In [1]:
import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise




def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
sensitivity = {1:0, 0.1:3, 0.01:6, 0.001:9, 0.5:1, 0.00005:13,
               0.0001:12, 0.00001:15, 0.000001:18, 0.0002:11, 0.05:4, 0.00002:14}

def set_position(p):
    if p >360 and p < -10:
        raise
    subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_move_p.py',
                                          str(p)])


In [3]:
import lakeshore

In [27]:
import time
import numpy as np
import lakeshore

#head = lakeshore.SSMSystem()
current_source = head.get_source_pod(1)
voltage_pod_1 = head.get_measure_pod(1)
voltage_pod_2 = head.get_measure_pod(2)
voltage_pod_3 = head.get_measure_pod(3)

# Configure current source pod
excitation_current_frequency = 17.3
excitation_current_amplitude = 150E-6
dc_current_level = 0.0
current_source.configure_i_range(autorange=False, max_level=excitation_current_amplitude)
current_source.use_dc_coupling()
current_source.apply_ac_current(excitation_current_frequency, excitation_current_amplitude, dc_current_level)


def lock_in_measure(pod, count=10, time_step=0.1, wait_before_measure=3, harm=2, sens=0.00001, time_constant=1.0):
    # Setup voltage measurement pod
    pod.setup_lock_in_measurement(reference_source='S1', time_constant=time_constant, reference_harmonic=harm)
    pod.configure_voltage_range(autorange=False, max_level=sens)

    time.sleep(wait_before_measure)

    xs, ys, rs, thetas = [], [], [], []
    for i in range(count):
        xs.append(pod.get_lock_in_x())
        ys.append(pod.get_lock_in_y())
        rs.append(pod.get_lock_in_magnitude())
        thetas.append(pod.get_lock_in_angle())

        time.sleep(time_step)

    x_mean, x_std = np.mean(xs), np.std(xs)
    y_mean, y_std = np.mean(ys), np.std(ys)
    r_mean, r_std = np.mean(rs), np.std(rs)
    theta_mean, theta_std = np.mean(thetas), np.std(thetas)

    return x_mean, y_mean, r_mean, theta_mean, \
           x_std, y_std, r_std, theta_std


# Example calls:
res1 = lock_in_measure(voltage_pod_1, harm=1, sens=0.01)
res3 = lock_in_measure(voltage_pod_3, harm=2, sens=0.01)

#Check 6221 compliance, Check 2400 output off state

In [16]:
res1

(0.04069341545936058,
 -9.765210180337539e-05,
 0.038269783700157295,
 -0.0024953103718271478,
 0.018920124884168703,
 4.6272405383153214e-05,
 0.01795202541943751,
 7.060433596716998e-05)

In [17]:
res3

(-6.495659873264239e-08,
 1.8029670214524412e-06,
 1.8041505662701964e-06,
 1.6069760649813378,
 4.284191185689004e-09,
 3.013768906904261e-09,
 2.8457500887928906e-09,
 0.0020937138280751966)

In [32]:
current_source.apply_ac_current(excitation_current_frequency, 0, dc_current_level)

In [18]:
import datetime
print(datetime.datetime.now())

2020-02-11 14:25:30.366651


In [19]:
import os

In [20]:
print(os.getcwd())

C:\Users\Admin\Desktop\Sam files\_switch


In [5]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL10::INSTR')

In [8]:
lock.lock_in_read()

(-0.0010672429344, -2.7712251267e-06, 0.0010672467761, -179.85116577)

In [28]:
def lock_angular_measurement_set_p(positions, lock, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.001, wait_before_measure=30, 
                             lock_count=10, sample_id="Fe2O3",  
                             time_after_move=10, time_out_count = 60*5):
    filename = "./data/" + sample_id + "_{}.csv".format(int(time.time()))
    with open(filename, 'w') as f:
        f.write("x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,ac_I,POS\n")
    
    for pos in positions:
        set_position(pos)
        time.sleep(time_after_move)
        print("current position: {}".format(pos))
        res1 = lock_in_measure(voltage_pod_1, harm=1, sens=0.01)
        res2 = lock_in_measure(voltage_pod_3, harm=2, sens=0.01)

        with open(filename, 'a') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(*res1, *res2, input_I, pos))
    set_position(0)
    time.sleep(60)

In [29]:
do_task = True

In [209]:
#ps = list(range(-5, 15, 3)) + list(range(15, 165, 10)) + list(range(165, 195, 3)) + list(range(195, 346, 10))

In [14]:
#ps = list(range(-5, 346, 10))

In [23]:
ps = list(range(0,361,10))

In [24]:
len(ps)

37

In [30]:
do_task = True

In [31]:
#rxy measurment 14, 10
fields = [20000]
temp = 300
#set_T(temp)
#time.sleep(25*60)
if not do_task:
    raise
for f in fields:
    #set_Field(f, True)

    lock_angular_measurement_set_p(ps, _, sample_id="Triton_2nmPt_Fe2o3_30nm_test_150uA_rxy_alpha"+ str(temp) + "K" + str(f)+ "G", 
                               first_range=0.01, second_range=0.00001,wait_before_measure=60)
     
set_Field(0, True)
do_task=False

current position: 0
current position: 10
current position: 20
current position: 30
current position: 40
current position: 50
current position: 60
current position: 70
current position: 80
current position: 90
current position: 100
current position: 110
current position: 120
current position: 130
current position: 140
current position: 150
current position: 160
current position: 170
current position: 180
current position: 190
current position: 200
current position: 210
current position: 220
current position: 230
current position: 240
current position: 250
current position: 260
current position: 270
current position: 280
current position: 290
current position: 300
current position: 310
current position: 320
current position: 330
current position: 340
current position: 350
current position: 360
-0.000416718539782
current field -0.000416718539782
